# Hypothesis 1: Central Bank Reaction and Confidence Effects - Detailed Analysis

This notebook provides an in-depth analysis of Hypothesis 1, which tests whether strong central bank reactions to debt service burdens combined with negative confidence effects create a threshold beyond which QE increases long-term yields.

## Economic Theory

The hypothesis is based on the idea that:

1. **Central Bank Reaction (γ₁):** Strong policy responses to rising debt service burdens
2. **Confidence Effects (λ₂):** Negative confidence effects that amplify policy transmission
3. **Threshold Behavior:** Beyond a certain debt service threshold, the combination of strong reactions and negative confidence creates a regime where QE becomes counterproductive

## Methodology

We use Hansen's threshold regression model to detect structural breaks:

$$y_t = \alpha_1 + \beta_1 X_t + \epsilon_t \quad \text{if } q_t \leq \gamma$$
$$y_t = \alpha_2 + \beta_2 X_t + \epsilon_t \quad \text{if } q_t > \gamma$$

Where:
- $y_t$ = Long-term yields (10Y Treasury)
- $X_t$ = [Central bank reaction, Confidence effects, Interaction term]
- $q_t$ = Debt service burden (threshold variable)
- $\gamma$ = Threshold parameter (estimated)

## Setup and Data Collection

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# QEIR imports
from qeir.core.hypothesis_testing import QEHypothesisTester, HypothesisTestingConfig
from qeir.utils.hypothesis_data_collector import HypothesisDataCollector
from qeir.core.models import HansenThresholdRegression

# Plotting setup
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (14, 10)
sns.set_palette("husl")

print("Hypothesis 1 Detailed Analysis - Setup Complete")

In [ ]:
# Initialize framework (replace with your API key)
import os
FRED_API_KEY = os.getenv('FRED_API_KEY')
if not FRED_API_KEY:
    raise ValueError('FRED_API_KEY environment variable is required')

collector = HypothesisDataCollector(fred_api_key=FRED_API_KEY)

# Enhanced configuration for detailed analysis
config = HypothesisTestingConfig(
    start_date="2008-01-01",
    end_date="2023-12-31",
    confidence_level=0.95,
    bootstrap_iterations=500,  # More iterations for better precision
    h1_threshold_trim=0.15,
    h1_min_regime_size=10,
    h1_test_alternative_thresholds=True,
    enable_robustness_tests=True
)

tester = QEHypothesisTester(data_collector=collector, config=config)
print("Framework initialized for detailed Hypothesis 1 analysis")

## Data Collection and Preprocessing

In [ ]:
# Collect Hypothesis 1 specific data
print("Collecting Hypothesis 1 data...")
h1_data = collector.collect_hypothesis1_data(config.start_date, config.end_date)

print(f"\nCollected {len(h1_data)} data series:")
for series_name, series in h1_data.items():
    if series is not None and not series.empty:
        print(f"  {series_name}: {len(series)} observations ({series.index.min()} to {series.index.max()})")
    else:
        print(f"  {series_name}: No data")

# Validate data quality
quality_report = collector.validate_data_quality(h1_data)
print(f"\nData quality score: {quality_report['summary']['overall_quality_score']:.1f}/100")

## Exploratory Data Analysis

In [ ]:
# Create comprehensive data visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Key variables for Hypothesis 1
key_vars = {
    'fed_total_assets': 'Central Bank Reaction (Fed Assets, $T)',
    'consumer_confidence': 'Confidence Effects (Consumer Sentiment)',
    'federal_interest_payments': 'Debt Service Burden (Interest Payments)',
    'treasury_10y': 'Long-term Yields (10Y Treasury, %)'
}

for i, (var_name, title) in enumerate(key_vars.items()):
    row, col = i // 2, i % 2
    
    if var_name in h1_data and h1_data[var_name] is not None:
        series = h1_data[var_name].dropna()
        axes[row, col].plot(series.index, series.values, linewidth=2, color=f'C{i}')
        axes[row, col].set_title(title, fontsize=12, fontweight='bold')
        axes[row, col].grid(True, alpha=0.3)
        axes[row, col].tick_params(axis='x', rotation=45)
        
        # Add QE period shading
        qe_periods = [
            ('2008-11-01', '2010-06-30', 'QE1'),
            ('2010-11-01', '2011-06-30', 'QE2'),
            ('2012-09-01', '2014-10-31', 'QE3')
        ]
        
        for start, end, label in qe_periods:
            axes[row, col].axvspan(pd.to_datetime(start), pd.to_datetime(end), 
                                 alpha=0.2, color='red', label=label if i == 0 else "")
    else:
        axes[row, col].text(0.5, 0.5, 'Data Not Available', 
                          ha='center', va='center', transform=axes[row, col].transAxes)
        axes[row, col].set_title(title, fontsize=12, fontweight='bold')

# Add legend for QE periods
if 'fed_total_assets' in h1_data:
    axes[0, 0].legend(loc='upper left')

plt.tight_layout()
plt.suptitle('Hypothesis 1: Key Variables Over Time', fontsize=16, fontweight='bold', y=1.02)
plt.show()

In [ ]:
# Correlation analysis
print("Correlation Analysis of Key Variables:")
print("=" * 50)

# Create correlation matrix
corr_data = {}
for var_name, title in key_vars.items():
    if var_name in h1_data and h1_data[var_name] is not None:
        corr_data[title.split('(')[0].strip()] = h1_data[var_name]

if len(corr_data) >= 2:
    corr_df = pd.DataFrame(corr_data).dropna()
    correlation_matrix = corr_df.corr()
    
    # Plot correlation heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='RdBu_r', center=0, 
                square=True, fmt='.3f', cbar_kws={'label': 'Correlation Coefficient'})
    plt.title('Correlation Matrix: Hypothesis 1 Variables', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("\nCorrelation Matrix:")
    print(correlation_matrix.round(3))
else:
    print("Insufficient data for correlation analysis")

## Threshold Analysis Preparation

In [ ]:
# Prepare data for threshold analysis
print("Preparing data for threshold analysis...")

# Load complete hypothesis data
data = tester.load_data()

# Extract key variables
if (data.long_term_yields is not None and data.central_bank_reaction is not None and 
    data.confidence_effects is not None and data.debt_service_burden is not None):
    
    # Align data to common dates
    common_dates = (data.long_term_yields.dropna().index
                   .intersection(data.central_bank_reaction.dropna().index)
                   .intersection(data.confidence_effects.dropna().index)
                   .intersection(data.debt_service_burden.dropna().index))
    
    print(f"Common observations: {len(common_dates)}")
    print(f"Date range: {common_dates.min()} to {common_dates.max()}")
    
    if len(common_dates) >= 50:
        # Extract aligned series
        y = data.long_term_yields.loc[common_dates]
        cb_reaction = data.central_bank_reaction.loc[common_dates]
        confidence = data.confidence_effects.loc[common_dates]
        threshold_var = data.debt_service_burden.loc[common_dates]
        
        # Create interaction term
        interaction = cb_reaction * confidence
        
        # Standardize variables for better numerical stability
        cb_reaction_std = (cb_reaction - cb_reaction.mean()) / cb_reaction.std()
        confidence_std = (confidence - confidence.mean()) / confidence.std()
        interaction_std = (interaction - interaction.mean()) / interaction.std()
        
        print("\nVariable Statistics:")
        print(f"Long-term yields: {y.mean():.3f} ± {y.std():.3f}")
        print(f"Central bank reaction: {cb_reaction.mean():.1f} ± {cb_reaction.std():.1f}")
        print(f"Confidence effects: {confidence.mean():.1f} ± {confidence.std():.1f}")
        print(f"Debt service burden: {threshold_var.mean():.1f} ± {threshold_var.std():.1f}")
        
        data_ready = True
    else:
        print("Insufficient overlapping data for analysis")
        data_ready = False
else:
    print("Required data series not available")
    data_ready = False

## Threshold Detection and Visualization

In [ ]:
if data_ready:
    # Visualize threshold variable vs dependent variable
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Scatter plot: Threshold variable vs Long-term yields
    axes[0].scatter(threshold_var, y, alpha=0.6, s=50)
    axes[0].set_xlabel('Debt Service Burden')
    axes[0].set_ylabel('Long-term Yields (%)')
    axes[0].set_title('Threshold Variable vs Dependent Variable')
    axes[0].grid(True, alpha=0.3)
    
    # Add trend line
    z = np.polyfit(threshold_var, y, 1)
    p = np.poly1d(z)
    axes[0].plot(threshold_var, p(threshold_var), "r--", alpha=0.8, linewidth=2)
    
    # Time series plot with threshold variable
    ax1 = axes[1]
    ax2 = ax1.twinx()
    
    line1 = ax1.plot(common_dates, y, 'b-', linewidth=2, label='Long-term Yields')
    line2 = ax2.plot(common_dates, threshold_var, 'r-', linewidth=2, alpha=0.7, label='Debt Service Burden')
    
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Long-term Yields (%)', color='b')
    ax2.set_ylabel('Debt Service Burden', color='r')
    ax1.tick_params(axis='y', labelcolor='b')
    ax2.tick_params(axis='y', labelcolor='r')
    ax1.set_title('Time Series: Yields and Threshold Variable')
    ax1.grid(True, alpha=0.3)
    
    # Combined legend
    lines = line1 + line2
    labels = [l.get_label() for l in lines]
    ax1.legend(lines, labels, loc='upper left')
    
    plt.tight_layout()
    plt.show()
    
    # Distribution of threshold variable
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.hist(threshold_var, bins=20, alpha=0.7, edgecolor='black')
    plt.xlabel('Debt Service Burden')
    plt.ylabel('Frequency')
    plt.title('Distribution of Threshold Variable')
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    stats.probplot(threshold_var, dist="norm", plot=plt)
    plt.title('Q-Q Plot: Threshold Variable Normality')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("Cannot proceed with threshold analysis due to data issues")

## Hansen Threshold Regression Analysis

In [ ]:
if data_ready:
    print("Running Hansen Threshold Regression...")
    
    # Run the full hypothesis test
    h1_results = tester.test_hypothesis1(data)
    
    print("\n" + "="*60)
    print("HANSEN THRESHOLD REGRESSION RESULTS")
    print("="*60)
    
    if 'error' not in h1_results.main_result:
        main_result = h1_results.main_result
        stat_sig = h1_results.statistical_significance
        
        print(f"\nModel Status: {'✓ FITTED' if main_result.get('threshold_detected') else '✗ NO THRESHOLD'}")
        print(f"Observations: {h1_results.data_period.get('observations')}")
        print(f"Sample Period: {h1_results.data_period.get('start_date')} to {h1_results.data_period.get('end_date')}")
        
        if main_result.get('threshold_detected'):
            print(f"\nThreshold Estimate: {main_result.get('threshold_value'):.4f}")
            print(f"Regime 1 Observations: {main_result.get('regime1_observations')} ({main_result.get('regime1_observations')/h1_results.data_period.get('observations', 1)*100:.1f}%)")
            print(f"Regime 2 Observations: {main_result.get('regime2_observations')} ({main_result.get('regime2_observations')/h1_results.data_period.get('observations', 1)*100:.1f}%)")
            
            print(f"\nModel Fit:")
            print(f"Overall R²: {main_result.get('overall_r2', 0):.4f}")
            print(f"Regime 1 R²: {main_result.get('regime1_r2', 0):.4f}")
            print(f"Regime 2 R²: {main_result.get('regime2_r2', 0):.4f}")
            
            if stat_sig:
                print(f"\nStructural Break Test:")
                print(f"F-statistic: {stat_sig.get('structural_break_fstat', 0):.4f}")
                print(f"P-value: {stat_sig.get('structural_break_pvalue', 1):.4f}")
                print(f"Significant at 5%: {'Yes' if stat_sig.get('structural_break_significant', False) else 'No'}")
            
            # Extract detailed model results if available
            if h1_results.model_results.hansen_results:
                hansen_res = h1_results.model_results.hansen_results
                
                if 'regime1_coefficients' in hansen_res and 'regime2_coefficients' in hansen_res:
                    print(f"\nRegime Coefficients:")
                    print(f"Regime 1 (Below Threshold):")
                    coef_names = ['Constant', 'CB Reaction', 'Confidence', 'Interaction']
                    for i, (name, coef) in enumerate(zip(coef_names, hansen_res['regime1_coefficients'])):
                        se = hansen_res.get('regime1_std_errors', [0]*len(coef_names))[i]
                        print(f"  {name}: {coef:.4f} (SE: {se:.4f})")
                    
                    print(f"\nRegime 2 (Above Threshold):")
                    for i, (name, coef) in enumerate(zip(coef_names, hansen_res['regime2_coefficients'])):
                        se = hansen_res.get('regime2_std_errors', [0]*len(coef_names))[i]
                        print(f"  {name}: {coef:.4f} (SE: {se:.4f})")
        else:
            print("\nNo significant threshold detected.")
            print("The relationship appears to be linear across the sample.")
    else:
        print(f"\n✗ Analysis failed: {h1_results.main_result['error']}")
else:
    print("Cannot run Hansen regression due to data preparation issues")

## Threshold Visualization and Regime Analysis

In [ ]:
if data_ready and 'h1_results' in locals() and 'error' not in h1_results.main_result:
    if h1_results.main_result.get('threshold_detected'):
        threshold_value = h1_results.main_result.get('threshold_value')
        
        # Create regime-based visualization
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        
        # Regime classification
        regime1_mask = threshold_var <= threshold_value
        regime2_mask = threshold_var > threshold_value
        
        # Plot 1: Scatter plot with regime coloring
        axes[0, 0].scatter(threshold_var[regime1_mask], y[regime1_mask], 
                          alpha=0.7, s=50, color='blue', label=f'Regime 1 (≤{threshold_value:.3f})')
        axes[0, 0].scatter(threshold_var[regime2_mask], y[regime2_mask], 
                          alpha=0.7, s=50, color='red', label=f'Regime 2 (>{threshold_value:.3f})')
        axes[0, 0].axvline(threshold_value, color='black', linestyle='--', linewidth=2, label='Threshold')
        axes[0, 0].set_xlabel('Debt Service Burden')
        axes[0, 0].set_ylabel('Long-term Yields (%)')
        axes[0, 0].set_title('Threshold Regression: Regime Classification')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        # Plot 2: Time series with regime shading
        axes[0, 1].plot(common_dates, y, 'k-', linewidth=2, label='Long-term Yields')
        
        # Shade regime periods
        for i in range(len(common_dates)):
            if regime1_mask.iloc[i]:
                axes[0, 1].axvspan(common_dates[i], common_dates[i], alpha=0.3, color='blue')
            else:
                axes[0, 1].axvspan(common_dates[i], common_dates[i], alpha=0.3, color='red')
        
        axes[0, 1].set_xlabel('Date')
        axes[0, 1].set_ylabel('Long-term Yields (%)')
        axes[0, 1].set_title('Time Series with Regime Classification')
        axes[0, 1].grid(True, alpha=0.3)
        
        # Plot 3: Threshold variable over time
        axes[1, 0].plot(common_dates, threshold_var, 'g-', linewidth=2)
        axes[1, 0].axhline(threshold_value, color='black', linestyle='--', linewidth=2, label='Threshold')
        axes[1, 0].fill_between(common_dates, threshold_var.min(), threshold_value, 
                               alpha=0.3, color='blue', label='Regime 1')
        axes[1, 0].fill_between(common_dates, threshold_value, threshold_var.max(), 
                               alpha=0.3, color='red', label='Regime 2')
        axes[1, 0].set_xlabel('Date')
        axes[1, 0].set_ylabel('Debt Service Burden')
        axes[1, 0].set_title('Threshold Variable Over Time')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
        
        # Plot 4: Regime statistics
        regime_stats = pd.DataFrame({
            'Regime 1': [
                y[regime1_mask].mean(),
                cb_reaction[regime1_mask].mean(),
                confidence[regime1_mask].mean(),
                interaction[regime1_mask].mean()
            ],
            'Regime 2': [
                y[regime2_mask].mean(),
                cb_reaction[regime2_mask].mean(),
                confidence[regime2_mask].mean(),
                interaction[regime2_mask].mean()
            ]
        }, index=['Yields', 'CB Reaction', 'Confidence', 'Interaction'])
        
        regime_stats.plot(kind='bar', ax=axes[1, 1], color=['blue', 'red'], alpha=0.7)
        axes[1, 1].set_title('Average Values by Regime')
        axes[1, 1].set_ylabel('Average Value')
        axes[1, 1].tick_params(axis='x', rotation=45)
        axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.suptitle('Hypothesis 1: Threshold Analysis Results', fontsize=16, fontweight='bold', y=1.02)
        plt.show()
        
        # Print regime statistics
        print("\nRegime Statistics:")
        print("=" * 40)
        print(regime_stats.round(3))
        
        print(f"\nRegime Transition Dates:")
        regime_changes = []
        current_regime = 1 if regime1_mask.iloc[0] else 2
        
        for i in range(1, len(regime1_mask)):
            new_regime = 1 if regime1_mask.iloc[i] else 2
            if new_regime != current_regime:
                regime_changes.append((common_dates[i], current_regime, new_regime))
                current_regime = new_regime
        
        if regime_changes:
            for date, from_regime, to_regime in regime_changes[:10]:  # Show first 10 transitions
                print(f"  {date.strftime('%Y-%m-%d')}: Regime {from_regime} → Regime {to_regime}")
            if len(regime_changes) > 10:
                print(f"  ... and {len(regime_changes) - 10} more transitions")
        else:
            print("  No regime transitions detected")
    else:
        print("No threshold detected - cannot create regime visualizations")
else:
    print("Cannot create threshold visualizations due to analysis issues")

## Economic Interpretation and Policy Implications

In [ ]:
if data_ready and 'h1_results' in locals() and 'error' not in h1_results.main_result:
    print("ECONOMIC INTERPRETATION")
    print("=" * 50)
    
    if h1_results.main_result.get('threshold_detected'):
        threshold_value = h1_results.main_result.get('threshold_value')
        
        print(f"\n1. THRESHOLD IDENTIFICATION:")
        print(f"   • Debt service threshold: {threshold_value:.4f}")
        print(f"   • This represents the critical level where QE transmission changes")
        
        if h1_results.statistical_significance.get('structural_break_significant'):
            print(f"   • The threshold is statistically significant (p < 0.05)")
        else:
            print(f"   • The threshold is not statistically significant")
        
        print(f"\n2. REGIME CHARACTERISTICS:")
        regime1_obs = h1_results.main_result.get('regime1_observations', 0)
        regime2_obs = h1_results.main_result.get('regime2_observations', 0)
        total_obs = regime1_obs + regime2_obs
        
        print(f"   • Regime 1 (Low debt service): {regime1_obs} observations ({regime1_obs/total_obs*100:.1f}%)")
        print(f"   • Regime 2 (High debt service): {regime2_obs} observations ({regime2_obs/total_obs*100:.1f}%)")
        
        print(f"\n3. POLICY IMPLICATIONS:")
        print(f"   • When debt service burden is below {threshold_value:.4f}:")
        print(f"     - QE transmission follows 'normal' patterns")
        print(f"     - Central bank reactions and confidence effects work as expected")
        
        print(f"   • When debt service burden exceeds {threshold_value:.4f}:")
        print(f"     - QE transmission may become counterproductive")
        print(f"     - Strong central bank reactions combined with negative confidence")
        print(f"       effects may increase rather than decrease long-term yields")
        
        print(f"\n4. HISTORICAL CONTEXT:")
        if 'regime_changes' in locals() and regime_changes:
            print(f"   • The economy switched between regimes {len(regime_changes)} times")
            print(f"   • This suggests the threshold mechanism is active and relevant")
        
        # Calculate time spent in each regime
        if data_ready:
            regime1_pct = (regime1_mask.sum() / len(regime1_mask)) * 100
            regime2_pct = (regime2_mask.sum() / len(regime2_mask)) * 100
            
            print(f"   • Time in Regime 1: {regime1_pct:.1f}% of sample period")
            print(f"   • Time in Regime 2: {regime2_pct:.1f}% of sample period")
        
        print(f"\n5. ROBUSTNESS AND LIMITATIONS:")
        print(f"   • Model R²: {h1_results.main_result.get('overall_r2', 0):.4f}")
        print(f"   • Results depend on data quality and model specification")
        print(f"   • Threshold may vary with different economic conditions")
        print(f"   • Consider alternative confidence and reaction measures")
        
    else:
        print(f"\n1. NO THRESHOLD DETECTED:")
        print(f"   • The relationship between variables appears linear")
        print(f"   • No evidence of regime-switching behavior")
        print(f"   • QE transmission may be stable across debt service levels")
        
        print(f"\n2. POSSIBLE EXPLANATIONS:")
        print(f"   • Debt service burden may not be the correct threshold variable")
        print(f"   • Threshold effects may be too subtle to detect")
        print(f"   • Sample period may not include sufficient variation")
        print(f"   • Other factors may dominate the relationship")
        
        print(f"\n3. ALTERNATIVE APPROACHES:")
        print(f"   • Try different threshold variables (e.g., debt-to-GDP ratio)")
        print(f"   • Use alternative confidence measures")
        print(f"   • Consider time-varying parameters")
        print(f"   • Examine sub-periods separately")
    
    print(f"\n" + "=" * 50)
else:
    print("Cannot provide economic interpretation due to analysis issues")

## Robustness Tests and Sensitivity Analysis

In [ ]:
if config.enable_robustness_tests and 'h1_results' in locals():
    print("ROBUSTNESS TESTS")
    print("=" * 40)
    
    # Run robustness tests
    print("Running robustness tests...")
    robustness_results = tester.run_robustness_tests({'hypothesis1': h1_results})
    
    if robustness_results:
        print("\n✓ Robustness tests completed")
        
        # Alternative periods
        if 'alternative_periods' in robustness_results:
            print("\nAlternative Time Periods:")
            for period_name, period_results in robustness_results['alternative_periods'].items():
                if 'error' not in period_results:
                    fitted = period_results.get('hypothesis1_fitted', False)
                    threshold = period_results.get('threshold_value', 'N/A')
                    obs = period_results.get('observations', 'N/A')
                    period = period_results.get('period', 'Unknown')
                    
                    status = "✓" if fitted else "✗"
                    print(f"  {status} {period}: Threshold={threshold}, Obs={obs}")
                else:
                    print(f"  ✗ {period_results.get('period', 'Unknown')}: {period_results['error']}")
        
        # Alternative specifications
        if 'alternative_specifications' in robustness_results:
            print("\nAlternative Specifications:")
            for spec_name, spec_results in robustness_results['alternative_specifications'].items():
                print(f"  • {spec_name}: {spec_results.get('specification', 'Unknown')}")
    else:
        print("No robustness test results available")
    
    # Manual sensitivity analysis
    print("\nSensitivity Analysis:")
    print("-" * 30)
    
    if data_ready:
        # Test different trimming parameters
        trim_values = [0.10, 0.15, 0.20, 0.25]
        sensitivity_results = []
        
        for trim in trim_values:
            try:
                temp_config = HypothesisTestingConfig(
                    start_date=config.start_date,
                    end_date=config.end_date,
                    h1_threshold_trim=trim,
                    bootstrap_iterations=50,  # Reduced for speed
                    enable_robustness_tests=False
                )
                
                temp_tester = QEHypothesisTester(data_collector=collector, config=temp_config)
                temp_results = temp_tester.test_hypothesis1(data)
                
                if 'error' not in temp_results.main_result:
                    threshold_detected = temp_results.main_result.get('threshold_detected', False)
                    threshold_value = temp_results.main_result.get('threshold_value', 'N/A')
                    
                    sensitivity_results.append({
                        'trim': trim,
                        'threshold_detected': threshold_detected,
                        'threshold_value': threshold_value
                    })
                    
                    status = "✓" if threshold_detected else "✗"
                    print(f"  Trim {trim:.2f}: {status} Threshold = {threshold_value}")
                else:
                    print(f"  Trim {trim:.2f}: ✗ Failed")
                    
            except Exception as e:
                print(f"  Trim {trim:.2f}: ✗ Error - {str(e)[:50]}...")
        
        # Summary of sensitivity
        if sensitivity_results:
            detected_count = sum(1 for r in sensitivity_results if r['threshold_detected'])
            print(f"\nSensitivity Summary:")
            print(f"  Threshold detected in {detected_count}/{len(sensitivity_results)} specifications")
            
            if detected_count > 0:
                threshold_values = [r['threshold_value'] for r in sensitivity_results if r['threshold_detected']]
                if threshold_values and all(isinstance(tv, (int, float)) for tv in threshold_values):
                    print(f"  Threshold range: {min(threshold_values):.4f} to {max(threshold_values):.4f}")
                    print(f"  Average threshold: {np.mean(threshold_values):.4f}")
else:
    print("Robustness tests disabled in configuration")

## Summary and Conclusions

In [ ]:
print("HYPOTHESIS 1 - SUMMARY AND CONCLUSIONS")
print("=" * 60)

if 'h1_results' in locals() and 'error' not in h1_results.main_result:
    print(f"\n📊 ANALYSIS SUMMARY:")
    print(f"   • Sample Period: {h1_results.data_period.get('start_date')} to {h1_results.data_period.get('end_date')}")
    print(f"   • Observations: {h1_results.data_period.get('observations')}")
    print(f"   • Method: Hansen Threshold Regression")
    print(f"   • Bootstrap Iterations: {config.bootstrap_iterations}")
    
    threshold_detected = h1_results.main_result.get('threshold_detected', False)
    
    if threshold_detected:
        print(f"\n✅ KEY FINDINGS:")
        print(f"   • Threshold Effect: DETECTED")
        print(f"   • Threshold Value: {h1_results.main_result.get('threshold_value'):.4f}")
        print(f"   • Statistical Significance: {'Yes' if h1_results.statistical_significance.get('structural_break_significant') else 'No'}")
        print(f"   • Model Fit (R²): {h1_results.main_result.get('overall_r2', 0):.4f}")
        
        print(f"\n🎯 ECONOMIC IMPLICATIONS:")
        print(f"   • QE transmission exhibits non-linear behavior")
        print(f"   • Debt service burden acts as a critical threshold")
        print(f"   • Policy effectiveness depends on fiscal conditions")
        print(f"   • Confidence effects amplify threshold behavior")
        
        print(f"\n⚠️  POLICY WARNINGS:")
        print(f"   • QE may become counterproductive above threshold")
        print(f"   • Monitor debt service burden carefully")
        print(f"   • Consider fiscal-monetary policy coordination")
        
    else:
        print(f"\n❌ KEY FINDINGS:")
        print(f"   • Threshold Effect: NOT DETECTED")
        print(f"   • Relationship appears linear")
        print(f"   • No evidence of regime switching")
        
        print(f"\n🤔 POSSIBLE EXPLANATIONS:")
        print(f"   • Threshold variable may be incorrect")
        print(f"   • Effects may be too subtle to detect")
        print(f"   • Sample period limitations")
        print(f"   • Model specification issues")
        
        print(f"\n🔄 RECOMMENDATIONS:")
        print(f"   • Try alternative threshold variables")
        print(f"   • Use different confidence measures")
        print(f"   • Extend sample period if possible")
        print(f"   • Consider alternative methodologies")
    
    print(f"\n📈 DATA QUALITY:")
    if 'quality_report' in locals():
        print(f"   • Overall Quality Score: {quality_report['summary']['overall_quality_score']:.1f}/100")
        print(f"   • Critical Issues: {quality_report['summary']['critical_issues']}")
        print(f"   • Data Reliability: {'High' if quality_report['summary']['overall_quality_score'] > 80 else 'Medium' if quality_report['summary']['overall_quality_score'] > 60 else 'Low'}")
    
    print(f"\n🔬 ROBUSTNESS:")
    if config.enable_robustness_tests:
        print(f"   • Robustness tests: COMPLETED")
        print(f"   • Alternative periods tested")
        print(f"   • Sensitivity analysis performed")
    else:
        print(f"   • Robustness tests: SKIPPED (for speed)")
        print(f"   • Enable for full analysis")
    
    print(f"\n📚 NEXT STEPS:")
    print(f"   1. Review results in context of economic theory")
    print(f"   2. Compare with existing literature")
    print(f"   3. Consider policy implications")
    print(f"   4. Proceed to Hypothesis 2 and 3 testing")
    print(f"   5. Generate publication-ready outputs")
    
else:
    print(f"\n❌ ANALYSIS FAILED")
    if 'h1_results' in locals():
        print(f"   Error: {h1_results.main_result.get('error', 'Unknown error')}")
    else:
        print(f"   Could not complete analysis due to data issues")
    
    print(f"\n🔧 TROUBLESHOOTING:")
    print(f"   • Check FRED API key")
    print(f"   • Verify internet connection")
    print(f"   • Review data quality issues")
    print(f"   • Try shorter time period")
    print(f"   • Reduce bootstrap iterations")

print(f"\n" + "=" * 60)
print(f"Hypothesis 1 detailed analysis completed.")
print(f"=" * 60)